In [95]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar as cl
pd.set_option('display.max_rows', 500)

In [96]:
#Reading CSV from current directory
df = pd.read_csv('hospitaldata.csv')

In [97]:
#Cleaning column names 
print(df.columns.values)

['Date' 'id' 'Time' 'Age' 'Sex' 'Consulting..Doctor' 'Specialty'
 'Procedure' 'Total..Charges' 'Amount..Received.' 'Amount..Balance'
 'Amount.Received.By' 'Amount.in.Hospital' 'Receptionist..Name' 'Next.Apt']


In [98]:
df.columns = df.columns.str.replace('.','')

In [99]:
#Convert object to datetime
df.Date=pd.to_datetime(df.Date)

In [100]:
#Cleaning Time column
df.Time[df.Time == '-'] = np.nan

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [101]:
#Cleaning column to upper case as it having one small 'f'
df[df.Sex == '-'] = np.nan
df.Sex = df.Sex.str.upper()

In [102]:
#Place nan wherever there is cancelled placed in total charges
df.TotalCharges[df.TotalCharges == 'Cancelled'] = np.nan
df.TotalCharges = df.TotalCharges.astype(float)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [103]:
df.Age.replace("[^0-9]",'',regex=True,inplace=True)
df.Age.replace("",np.nan,regex=True,inplace=True)
x = np.array(df.Age).astype(float)
print("Average Age is ",str(np.nanmean(x)))

Average Age is  32.5721649485


In [11]:
#Place nan wherever there is cancelled placed in Procedure
df.Procedure[df.Procedure == 'Cancelled'] = np.nan

C:\Users\Hammad Ali Khan\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [12]:
#(df.Date.groupby([df.Date]).count())
print("Most visited day of visitor was",str(cl.day_name[(df.Date.value_counts().idxmax()).weekday()]))

Most visited day of visitor was Monday


In [13]:
#4. How many children were entertained? (Make a Bracket of Age from 1-12)
a = x[pd.notnull(x)]
print("Number of childern entertained were",str(len(a[(a >= 1) & (a <= 12)])))

Number of childern entertained were 24


In [14]:
#5. Which gender type had what kind of procedure in abundance? i.e. Female visit mostly because of Gynae Problem 
bytreatement = df.groupby(df.Sex)
bytreatement['Procedure'].describe()

Sex        
F    count              100
     unique              29
     top       Consultation
     freq                46
M    count               98
     unique              28
     top       Consultation
     freq                34
Name: Procedure, dtype: object

In [15]:
#6. Which Doctor is earning highest? 
bydoctor = df.groupby(df.ConsultingDoctor)
print("Highest paid doctor is",str(bydoctor['TotalCharges'].aggregate(np.sum).idxmax()))

Highest paid doctor is Dr Alaf Khan


In [16]:
#7. Which procedure type earns more money? 
byProcedure = df.groupby(df.Procedure)
print("Highest earning procedure is",str(byProcedure['TotalCharges'].sum().idxmax()),"worth",str(byProcedure['TotalCharges'].sum().max()))

Highest earning procedure is Orthodontics worth 240000.0


In [35]:
#8. Which time of the day has highest frequency of visits by hour?
print(str(int(df.Time.dt.hour.value_counts().idxmax())),"have highest visits of",str(df.Time.dt.hour.value_counts().max()))

13 have highest visits of 29


In [40]:
#Q9 Create a bracket of time by Morning, Afternoon, Evening, Night (6am – 12pm – Morning, 12 pm- 4 pm, Afternoon, 4 pm- 7pm, Evening, 7pm – 6 am, Night).
def bracket(hours):
    if  hours >= 6 and hours < 12:
        return "Morning"
    elif hours >= 12 and hours < 14:
        return "Afternoon"
    elif  hours >= 14 and hours < 19:
        return "Evening"
    elif  hours >= 19 and hours <= 23 or hours >= 0 and hours < 6 :
        return "Night"
    else:
        return np.NAN
    
df['Time_brackets']=df.Time.dt.hour.apply(bracket)
print(df.Time_brackets)

0        Morning
1        Morning
2      Afternoon
3      Afternoon
4        Evening
5        Evening
6        Evening
7        Evening
8        Evening
9        Evening
10       Evening
11       Evening
12     Afternoon
13       Evening
14       Evening
15         Night
16     Afternoon
17         Night
18         Night
19     Afternoon
20       Evening
21       Evening
22     Afternoon
23     Afternoon
24     Afternoon
25           NaN
26         Night
27           NaN
28     Afternoon
29     Afternoon
30       Evening
31       Evening
32       Evening
33       Evening
34       Evening
35       Evening
36           NaN
37           NaN
38       Evening
39       Evening
40       Evening
41       Morning
42       Evening
43       Evening
44       Morning
45       Evening
46         Night
47       Evening
48       Evening
49         Night
50           NaN
51     Afternoon
52       Evening
53       Evening
54       Morning
55       Morning
56     Afternoon
57       Evening
58       Eveni

In [21]:
#10. How many patients are repeated visitors?
array = pd.value_counts(df['id'].values, sort=True)
print("Number of repeated patients are ",str(len(array[array > 1])))

Number of repeated patients are  37


In [53]:
#11. Give us the id of repeated visitors.
print("Id of repeated visitors are : ")
array[array > 1]

Id of repeated visitors are : 


1.0      12
122.0     5
46.0      5
94.0      4
17.0      4
140.0     4
107.0     3
132.0     3
63.0      3
145.0     3
45.0      3
114.0     3
101.0     3
109.0     3
40.0      2
12.0      2
150.0     2
64.0      2
13.0      2
88.0      2
97.0      2
80.0      2
151.0     2
59.0      2
149.0     2
116.0     2
118.0     2
112.0     2
25.0      2
20.0      2
96.0      2
153.0     2
120.0     2
133.0     2
130.0     2
100.0     2
4.0       2
dtype: int64

In [20]:
#12. Which patients visited again for the same problem? 
bypatientProcedure = df.groupby(df.id)
patients_repeat = bypatientProcedure['Procedure'].size()[bypatientProcedure['Procedure'].size() > 1]
patients_repeat

id
1.0      12
4.0       2
12.0      2
13.0      2
17.0      4
20.0      2
25.0      2
40.0      2
45.0      3
46.0      5
59.0      2
63.0      3
64.0      2
80.0      2
88.0      2
94.0      4
96.0      2
97.0      2
100.0     2
101.0     3
107.0     3
109.0     3
112.0     2
114.0     3
116.0     2
118.0     2
120.0     2
122.0     5
130.0     2
132.0     3
133.0     2
140.0     4
145.0     3
149.0     2
150.0     2
151.0     2
153.0     2
dtype: int64

In [109]:
# 13. What is the median age for Females and Males? 
df.Age = pd.to_numeric(df.Age)
df.groupby('Sex')['Age'].median()

Sex
F    30.0
M    29.0
Name: Age, dtype: float64

In [124]:
#14. What is the total amount in balance? 
df.replace(to_replace=df['AmountBalance'].unique()[0], value=np.nan, inplace=True)
df['AmountBalance']=df['AmountBalance'].map(lambda x: str(x).replace(',',''))
df['AmountBalance'] = df['AmountBalance'].astype(float)
print(df['AmountBalance'].sum())

222500.0


In [128]:
#15. How much money was made by Procedure Type “Consultation”? 
print("Money made by Consultation is",df.TotalCharges[df.Procedure == "Consultation"].sum())

Money made by Consultation is 82450.0


In [139]:
#16.  Is there a relation between Age and Total Charges paid? 
print("Its shows that it has positive relation.")
df.corr().loc[['Age'],['TotalCharges']]

Its shows that it has positive relation.


,TotalCharges
Age,0.029521


In [145]:
#17. Which Age group had highest number of visits? 
print("Age group of",str(int(df.Age.value_counts().idxmax())))

Age group of 30


In [148]:
#18. What is the total cost earned by Procedure Type X Ray and Scalling together? 
print("Total cost is ",str(df[(df.Procedure== 'X Rays') | (df.Procedure == 'Scalling')]['TotalCharges'].sum()))

Total cost is  16500.0


In [151]:
df.to_csv("CleanData.csv")